In [ ]:
# Extraction-based MRC를 이해하고 huggingface 라이브러리를 활용해 데이터 전처리와 모델 학습 및 평가

# 필요 패키지 설치

In [ ]:
!pip install datasets
!pip install evaluate
# !pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.7 MB/s eta 0:00:00


In [ ]:
# To use utility functions defined in examples.
!git clone https://github.com/huggingface/transformers.git

import sys
sys.path.append('transformers/examples/pytorch/question-answering')

Cloning into 'transformers'...
remote: Enumerating objects: 260643, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 260643 (delta 128), reused 94 (delta 94), pack-reused 260479 (from 3)
Receiving objects: 100% (260643/260643), 270.37 MiB | 9.76 MiB/s, done.
Resolving deltas: 100% (192523/192523), done.


# 데이터 및 평가 지표 불러오기

korquad dataset을 이용해 Extraction-based MRC.

metric은 squad에서 쓴 metric을 그대로 사용 가능

huggigngface datasets을 활용해 간편하게 dataset과 metric을 준비

In [ ]:
from datasets import load_dataset
from evaluate import load

datasets = load_dataset("squad_kor_v1")
metric = load("squad")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.29k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/11.6M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60407 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5774 [00:00<?, ? examples/s]

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 60407
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5774
    })
})

In [ ]:
datasets['train'][0]

{'id': '6566495-0-0',
 'title': '파우스트_서곡',
 'context': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.',
 'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?',
 'answers': {'text': ['교향곡'], 'answer_start': [54]}}

# Pre-trained 모델 및 토크나이저 호출

In [ ]:
from transformers import (
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer
)

model_name = "bert-base-multilingual-cased"

config = AutoConfig.from_pretrained(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

model = AutoModelForQuestionAnswering.from_pretrained(model_name, config=config)

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

# 설정에 쓰이는 값들

In [ ]:
max_seq_length = 384 # 질문과 컨텍스트, special token을 합한 문자열의 최대 길이
pad_to_max_length = True
doc_stride = 128 # 컨텍스트가 너무 길어서 나눴을 때 오버랩되는 시퀀스의 길이
max_train_samples = 16
max_val_samples = 16
preprocessing_num_workers = 4
batch_size = 4
num_train_epochs = 2
n_best_size = 20
max_answer_length = 30

# 전처리하기

주어진 텍스트를 토크나이징

이 때 텍스트의 길이가 max_seq_length를 넘으면 stride만큼 슬라이딩하며 여러 개로 쪼개는 과정

즉, 하나의 example에서 일부분이 겹치는 여러 sequence(feature)가 생길 수 있음

In [ ]:
def prepare_train_features(examples):
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second", # max_seq_length까지 truncate한다. pair의 두번째 파트(context)만 잘라냄.
        max_length=max_seq_length,
        stride=doc_stride,
        return_overflowing_tokens=True, # 길이를 넘어가는 토큰들을 반환할 것인지
        return_offsets_mapping=True, # 각 토큰에 대해 (char_start, char_end) 정보를 반환한 것인지
        padding="max_length",
    )

# example 하나가 여러 sequence에 대응하는 경우를 위해 매핑이 필요함.
    overflow_to_sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
# offset_mappings으로 토큰이 원본 context 내 몇번째 글자부터 몇번째 글자까지 해당하는지 알 수 있음.
    offset_mapping = tokenized_examples.pop("offset_mapping")

# 정답지를 만들기 위한 리스트
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

# 해당 example에 해당하는 sequence를 찾음.
        sequence_ids = tokenized_examples.sequence_ids(i)

# sequence가 속하는 example을 찾는다.
        example_index = overflow_to_sample_mapping[i]
        answers = examples["answers"][example_index]

# 텍스트에서 answer의 시작점, 끝점
        answer_start_offset = answers["answer_start"][0]
        answer_end_offset = answer_start_offset + len(answers["text"][0])

# 텍스트에서 현재 span의 시작 토큰 인덱스
        token_start_index = 0
        while sequence_ids[token_start_index] != 1:
            token_start_index += 1

# 텍스트에서 현재 span 끝 토큰 인덱스
        token_end_index = len(input_ids) - 1
        while sequence_ids[token_end_index] != 1:
            token_end_index -= 1

# answer가 현재 span을 벗어났는지 체크
        if not (offsets[token_start_index][0] <= answer_start_offset and offsets[token_end_index][1] >= answer_end_offset):
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
# token_start_index와 token_end_index를 answer의 시작점과 끝점으로 옮김
            while token_start_index < len(offsets) and offsets[token_start_index][0] <= answer_start_offset:
                token_start_index += 1
            tokenized_examples["start_positions"].append(token_start_index - 1)
            while offsets[token_end_index][1] >= answer_end_offset:
                token_end_index -= 1
            tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

# 전처리 함수를 train dataset에 적용하고 결과를 확인

In [ ]:
train_dataset = datasets["train"]
print(train_dataset[0])
# 몇 개만 sampling해서 전처리 됨을 확인합니다.

train_dataset = train_dataset.select(range(max_train_samples))
print(len(train_dataset))

{'id': '6566495-0-0', 'title': '파우스트_서곡', 'context': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.', 'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?', 'answers': {'text': ['교향곡'], 'answer_start': [54]}}
16


In [ ]:
column_names = datasets["train"].column_names
train_dataset = train_dataset.map(
            prepare_train_features,
            batched=True,
            num_proc=preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=True,
        )

train_dataset[0]

Map (num_proc=4):   0%|          | 0/16 [00:00<?, ? examples/s]

{'input_ids': [101,
  9318,
  78136,
  70162,
  11018,
  8905,
  119351,
  10459,
  9901,
  89108,
  101825,
  9642,
  11664,
  9294,
  119137,
  10622,
  9511,
  11664,
  13764,
  9965,
  11018,
  11287,
  136,
  102,
  16221,
  10954,
  9318,
  78136,
  70162,
  11018,
  8905,
  119351,
  10459,
  9901,
  89108,
  15184,
  10622,
  62849,
  9642,
  11664,
  8924,
  8996,
  24974,
  10530,
  9246,
  32158,
  10739,
  8973,
  26737,
  35756,
  9448,
  36210,
  11261,
  9960,
  12424,
  90387,
  8907,
  79544,
  55670,
  10622,
  9511,
  26737,
  11018,
  9153,
  10622,
  8854,
  40410,
  119,
  9638,
  9485,
  12310,
  9318,
  78136,
  70162,
  11018,
  16347,
  27056,
  9387,
  9088,
  119267,
  11467,
  9407,
  16617,
  15891,
  54918,
  9056,
  100,
  9414,
  65649,
  10739,
  17342,
  9686,
  58931,
  11882,
  9489,
  89292,
  10530,
  8843,
  118813,
  51491,
  9272,
  97146,
  12605,
  26444,
  119394,
  100929,
  11513,
  9248,
  49742,
  9901,
  89108,
  15184,
  10459,
  9491,

# validation set에서도 적용

validation set에서는 대신 정답을만드는 과정은 제외


In [ ]:
def prepare_validation_features(examples):
    tokenized_examples = tokenizer(
        examples['question'],
        examples['context'],
        truncation="only_second",
        max_length=max_seq_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1

        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [ ]:
# validation set에 validation set 전처리 함수를 적용

eval_examples = datasets["validation"]
eval_examples = eval_examples.select(range(max_val_samples))
eval_dataset = eval_examples.map(
            prepare_validation_features,
            batched=True,
            num_proc=preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=True)

Map (num_proc=4):   0%|          | 0/16 [00:00<?, ? examples/s]

# Fine-tuning하기

Fine-tuning 이후 나온 값이 원래의 context에 대응 될 수 있도록 후처리가 필요

함수를 활용해 기존 reference(정답)과 비교 가능하도록 후처리를 진행

In [ ]:
from transformers import default_data_collator, TrainingArguments, EvalPrediction
from trainer_qa import QuestionAnsweringTrainer
from utils_qa import postprocess_qa_predictions

def post_processing_function(examples, features, predictions):
    predictions = postprocess_qa_predictions(
        examples=examples,
        features=features,
        predictions=predictions,
        version_2_with_negative=False,
        n_best_size=n_best_size,
        max_answer_length=max_answer_length,
        null_score_diff_threshold=0.0,
        output_dir=training_args.output_dir,
    )


# Format the result to the format the metric expects.
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in predictions.items()]
    references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
    return EvalPrediction(predictions=formatted_predictions, label_ids=references)

# huggingface의 QuestionAnsweringTrainer을 활용해 Fine-tuning을 적용

In [ ]:
def compute_metrics(p: EvalPrediction):
    return metric.compute(predictions=p.predictions, references=p.label_ids)

In [ ]:
training_args = TrainingArguments(
    output_dir="outputs",
    do_train=True,
    do_eval=True,
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
)

trainer = QuestionAnsweringTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        eval_examples=datasets["validation"],
        tokenizer=tokenizer,
        data_collator=default_data_collator,
        post_process_function=post_processing_function,
        compute_metrics=compute_metrics,
    )

In [ ]:
train_result = trainer.train()

Step,Training Loss


In [ ]:
train_result

TrainOutput(global_step=12, training_loss=3.5000635782877603, metrics={'train_runtime': 277.6484, 'train_samples_per_second': 0.173, 'train_steps_per_second': 0.043, 'total_flos': 9406683242496.0, 'train_loss': 3.5000635782877603, 'epoch': 2.0})

In [ ]:
metrics = trainer.evaluate()

  0%|          | 0/5774 [00:00<?, ?it/s]

In [ ]:
metrics

{'eval_exact_match': 0.0,
 'eval_f1': 0.04975780867710206,
 'eval_runtime': 24.2691,
 'eval_samples_per_second': 0.659,
 'eval_steps_per_second': 0.165,
 'epoch': 2.0}

# 학습된 모델 불러오기

아학습된 모델을 불러와서 평가를 진행

In [ ]:
finetuned_model = AutoModelForQuestionAnswering.from_pretrained('sangrimlee/bert-base-multilingual-cased-korquad')

finetuned_model = finetuned_model.eval()

finetuned_trainer = QuestionAnsweringTrainer(
    model=finetuned_model,
    args=TrainingArguments(
        output_dir="finetuned_outputs",
        do_eval=True,
        per_device_eval_batch_size=batch_size,
    ),
    train_dataset=None,
    eval_dataset=eval_dataset,
    eval_examples=datasets["validation"],
    tokenizer=tokenizer,
    data_collator=default_data_collator,
    post_process_function=post_processing_function,
    compute_metrics=compute_metrics,
)

finetuned_metrics = finetuned_trainer.evaluate()
finetuned_metrics


/content/transformers/examples/pytorch/question-answering/trainer_qa.py:33: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `QuestionAnsweringTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


  0%|          | 0/5774 [00:00<?, ?it/s]

{'eval_exact_match': 0.22514721163837895,
 'eval_f1': 0.2575585135335741,
 'eval_model_preparation_time': 0.0031,
 'eval_runtime': 22.9405,
 'eval_samples_per_second': 0.697,
 'eval_steps_per_second': 0.174}

In [38]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

document = "질문: 세종대왕은 언제 태어났어? 답변:"
input_ids = tokenizer(document, return_tensors='pt').input_ids.to(device)
model = model.to(device)

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

TypeError: The current model class (BertForQuestionAnswering) is not compatible with `.generate()`, as it doesn't have a language model head. Classes that support generation often end in one of these names: ['ForCausalLM', 'ForConditionalGeneration', 'ForSpeechSeq2Seq', 'ForVision2Seq'].